In [40]:
import pandas as pd
import numpy as np
import networkx as nx
import tensorflow as tf
from tensorflow.keras import layers
from collections import deque
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, Embedding

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import tensorflow.keras.backend as K
from sklearn.preprocessing import LabelEncoder
import gym
from gym import spaces
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.sparse import coo_matrix
import scipy as sp

# Data preprocessing
# Load the Yoochoose dataset and preprocess it to create session-based sequences of interactions
data = pd.read_csv('yoochoose_dataset/filtered_clicks.dat',
                   names=['session_id', 'timestamp', 'item_id', 'category'],
                   dtype={'session_id': 'int64', 'timestamp': 'str', 'item_id': 'int64', 'category': 'int64'},
                   parse_dates=['timestamp'])

# Create item and session maps
item_map = dict(zip(np.unique(data.item_id), range(len(np.unique(data.item_id)))))
session_map = dict(zip(np.unique(data.session_id), range(len(np.unique(data.session_id)))))

# Map item and session IDs
data['item_id'] = data['item_id'].map(item_map)
data['session_id'] = data['session_id'].map(session_map)

# Sort by session and timestamp
data = data.sort_values(['session_id', 'timestamp'])

# Create next item and session columns
data['next_item_id'] = data.groupby('session_id')['item_id'].shift(-1)
data['next_session_id'] = data.groupby('session_id')['session_id'].shift(-1)
data = data.dropna()

# Convert data to numpy arrays
session_ids = data['session_id'].values.astype('int32')
item_ids = data['item_id'].values.astype('int32')
next_item_ids = data['next_item_id'].values.astype('int32')
next_session_ids = data['next_session_id'].values.astype('int32')
timestamps = data['timestamp'].values

# Create a directed graph
graph = nx.DiGraph()

# Add nodes to the graph
graph.add_nodes_from(item_map.values())

# Add edges between items that co-occur in the same session
for session_id in np.unique(session_ids):
    items_in_session = item_ids[session_ids == session_id]
    for i in range(len(items_in_session)):
        for j in range(i + 1, len(items_in_session)):
            if not graph.has_edge(items_in_session[i], items_in_session[j]):
                graph.add_edge(items_in_session[i], items_in_session[j], weight=0)
            graph[items_in_session[i]][items_in_session[j]]['weight'] += 1

# Normalize edge weights
for u, v, d in graph.edges(data=True):
    d['weight'] /= np.sqrt(graph.degree(u) * graph.degree(v))            

# Create adjacency matrix
adj_matrix = coo_matrix(nx.to_numpy_array(graph, weight='weight', dtype=np.float32))
adj_matrix = tf.sparse.SparseTensor(indices=np.array([adj_matrix.row, adj_matrix.col]).T,
                                    values=adj_matrix.data,
                                    dense_shape=adj_matrix.shape)    
print(adj_matrix.shape)
num_nodes = adj_matrix.shape[0]

(3007, 3007)


In [61]:
class GNN(tf.keras.layers.Layer):
    def __init__(self, num_hidden=16, num_layers=2, num_classes=2, **kwargs):
        super().__init__(**kwargs)
        self.num_hidden = num_hidden
        self.num_layers = num_layers
        self.num_classes = num_classes
        
        # define dense layers
        self.dense_layers = []
        for i in range(num_layers):
            self.dense_layers.append(tf.keras.layers.Dense(num_hidden, activation="relu"))
            
        self.embedding = tf.keras.layers.Embedding(input_dim=num_nodes, output_dim=num_hidden)
        
        # define final classification layer
        self.classification_layer = tf.keras.layers.Dense(num_classes, activation="softmax")

        
    def call(self, inputs, **kwargs):
        node_ids, adj_matrix = inputs
        
        # create node embeddings
        x = self.embedding(node_ids)
        
        # apply dense layers
        for layer in self.dense_layers:
            # Transpose the feature matrix before multiplying with the adjacency matrix
            x = layer(tf.transpose(x, perm=[0, 2, 1]))
            x = tf.transpose(x, perm=[0, 2, 1])
            
            # apply dropout
            x = tf.keras.layers.Dropout(0.5)(x, training=kwargs.get("training", False))
            
            # apply skip connection
            x = x + self.embedding(node_ids)
            
            # apply normalization
            x = tf.keras.layers.BatchNormalization()(x)
            
            # apply activation
            x = tf.keras.activations.relu(x)
            
            # multiply with adjacency matrix
            x = tf.sparse.sparse_dense_matmul(adj_matrix, x)
            
        return x

In [ ]:
# Define actor model
class Actor(tf.keras.Model):
    def __init__(self, num_actions, hidden_size):
        super(Actor, self).__init__()
        self.dense1 = layers.Dense(hidden_size, activation='relu')
        self.dropout = layers.Dropout(0.5)
        self.dense2 = layers.Dense(num_actions, activation='softmax')
    
    def call(self, inputs):
        embeddings = inputs
        x = self.dense1(embeddings)
        x = self.dropout(x)
        x = self.dense2(x)
        return x

In [ ]:
# Define DQN model
class DQN(tf.keras.Model):
    def __init__(self, num_actions, hidden_size):
        super(DQN, self).__init__()
        self.dense1 = layers.Dense(hidden_size, activation='relu')
        self.dropout = layers.Dropout(0.5)
        self.dense2 = layers.Dense(num_actions)
    
    def call(self, inputs):
        embeddings, action = inputs
        x = tf.concat([embeddings, action], axis=-1)
        x = self.dense1(x)
        x = self.dropout(x)
        x = self.dense2(x)
        return x

In [64]:
#define hyperparameters
batch_size = 64
learning_rate = 0.001
embedding_dim = 32
num_epochs = 10


# split data into train and validation
train_data, val_data = train_test_split(data, test_size=0.2)

# Define loss function and optimizer
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate)


# Initialize GNN model
gnn = GNN(num_items, embedding_dim)

# Training loop
for i in range(num_epochs):
    for j in range(0, len(train_data), batch_size):
        batch_data = train_data[j:j+batch_size]
        item_ids = batch_data['item_id'].values.astype('int32')
        # graph = nx.from_pandas_edgelist(batch_data, source='item_id', target='next_item_id', create_using=nx.DiGraph())
        # graph = nx.to_numpy_array(graph, dtype=np.float32)
        # graph = tf.convert_to_tensor(graph)
        with tf.GradientTape() as tape:
            embeddings = gnn([item_ids, graph])
            loss = loss_fn(graph, embeddings)
        gradients = tape.gradient(loss, gnn.trainable_variables)
        optimizer.apply_gradients(zip(gradients, gnn.trainable_variables))
for j in range(0, len(val_data), batch_size):
    batch_data = val_data[j:j+batch_size]
    item_ids = batch_data['item_id'].values.astype('int32')
    graph = nx.from_pandas_edgelist(batch_data, source='item_id', target='next_item_id', create_using=nx.DiGraph())
    graph = nx.to_numpy_array(graph, dtype=np.float32)
    graph = tf.convert_to_tensor(graph)
    embeddings = gnn([item_ids, graph])
    val_loss = loss_fn(graph, embeddings)
    total_val_loss += val_loss
val_loss = total_val_loss / len(val_data)


InvalidArgumentError: Exception encountered when calling layer 'gnn_12' (type GNN).

{{function_node __wrapped__Transpose_device_/job:localhost/replica:0/task:0/device:CPU:0}} transpose expects a vector of size 2. But input(1) is a vector of size 3 [Op:Transpose]

Call arguments received by layer 'gnn_12' (type GNN):
  • inputs=['tf.Tensor(shape=(64,), dtype=int32)', 'tf.Tensor(shape=(114, 114), dtype=float32)']
  • kwargs={'training': 'None'}

In [1]:
(2.5*-1.25)+(-2.5*-6.25)+(6.5*13.75)+(1.5*5.75)+(-12.5*-12.25)+(4.5*1.75)+(-1.5*-3.25)+(0.5*-0.25)

276.25

In [3]:
276.25/7

39.464285714285715

In [4]:
(2.5*0.725)+(-2.5*-0.375)+(6.5*-0.575)+(1.5*-0.475)+(-12.5*-0.275)+(4.5*0.225)+(-1.5*0.325)+(0.5*0.625)

2.575000000000001

In [5]:
2.575000000000001/7

0.367857142857143

In [6]:
(-1.25*0.725)+(-6.25*-0.375)+(13.75*-0.575)+(5.75*-0.475)+(-12.25*-0.275)+(1.75*0.225)+(-3.25*0.325)+(-0.25*0.625)

-6.6499999999999995

In [7]:
-6.6499999999999995/7

-0.95

In [8]:
2.5*2.5*2 + 6.5*6.5 + 1.5*1.5 + 12.5*12.5 + 4.5*4.5 + 1.5*1.5 + 0.5*0.5 

236.0

In [9]:
236/7

33.714285714285715

In [10]:
1.25*1.25 + 6.25*6.25 + 13.75*13.75 + 5.75*5.75 +12.25*12.25 + 1.75*1.75 + 3.25*3.25 + 0.25*0.25

426.5

In [11]:
426.5/7

60.92857142857143

In [12]:
0.725*0.725 + 0.375*0.375 + 0.575*0.575 + 0.475*0.475 + 0.275*0.275 + 0.225*0.225 + 0.325*0.325 + 0.625*0.625

1.845

In [13]:
1.845/7

0.26357142857142857

NameError: name 'sqr' is not defined